## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Sep. 15th Sep. 22nd


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,September-20-2024,Condo/Co-op,231 174th St #607,Sunny Isles Beach,FL,33160.0,308000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/23...,MARMLS,A11587905,N,Y,25.937475,-80.124967


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [10]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-08-26') & (df['SOLD DATE'] <= '2024-09-02')]

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
75,PAST SALE,2024-09-23,Condo/Co-op,301 174th St #1506,Sunny Isles Beach,FL,33160.0,320000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Beaches MLS,RX-10971897,N,Y,25.937377,-80.130117
83,PAST SALE,2024-09-23,Condo/Co-op,1330 West Ave #1806,Miami Beach,FL,33139.0,850000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1330-Wes...,MARMLS,A11606508,N,Y,25.784783,-80.143443
121,PAST SALE,2024-09-23,Condo/Co-op,6422 Collins Ave #901,Miami Beach,FL,33141.0,600000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6422-Col...,MARMLS,A11537418,N,Y,25.846484,-80.120521
18,PAST SALE,2024-09-23,Condo/Co-op,10839 NW 7th St Unit 12-21,Miami,FL,33172.0,299250.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/10839-NW-7th-S...,MARMLS,A11618022,N,Y,25.778064,-80.372267
93,PAST SALE,2024-09-21,Condo/Co-op,606 W 81st St #405,Hialeah,FL,33014.0,200000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/606-W-81st-S...,MARMLS,A11590372,N,Y,25.895969,-80.297233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,PAST SALE,2024-09-16,Condo/Co-op,1200 Brickell Bay Dr #3608,Miami,FL,33131.0,480000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1200-Brickell-...,MARMLS,A11581913,N,Y,25.761769,-80.189921
91,PAST SALE,2024-09-16,Condo/Co-op,7074 SW 114th Pl Unit C,Miami,FL,33173.0,442000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/7074-SW-114th-...,MARMLS,A11587384,N,Y,25.701679,-80.378186
101,PAST SALE,2024-09-16,Condo/Co-op,20191 E Country Club Dr #2403,Aventura,FL,33180.0,850000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20191-E-Countr...,MARMLS,A11560605,N,Y,25.963914,-80.125416
103,PAST SALE,2024-09-16,Condo/Co-op,21085 NE 34th Ave Unit 203-1,Aventura,FL,33180.0,400000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/21085-NE-34...,MARMLS,A11539553,N,Y,25.971433,-80.132424


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

134


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$684,642


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$503


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$91,741,989


In [14]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [21]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
107
Input Previous Week Condo Average Sales Price:
876980
Input Previous Week Condo Average PSF:
574
Input Previous Week Condo Sales total (ex: 198_000_000)
93_800_000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-16,Condo/Co-op,5775 Collins Ave #402,Miami Beach,FL,33140.0,5625000.0,2.0,3.0,AMD PL OF 2ND OCEAN FRONT,1612.0,NaN,2021.0,NaN,3489.0,4431.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5775-Col...,MARMLS,A11602237,N,Y,25.839257,-80.120395,https://www.redfin.com/FL/Miami-Beach/5775-Col...,Christopher Jude,"PGutman, LLC",None,None,Christopher Jude,"PGutman, LLC",Christopher Jude,"PGutman, LLC",1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-16,Condo/Co-op,5775 Collins Ave #402,Miami Beach,FL,33140.0,5625000.0,2.0,3.0,AMD PL OF 2ND OCEAN FRONT,1612.0,NaN,2021.0,NaN,3489.0,4431.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5775-Col...,MARMLS,A11602237,N,Y,25.839257,-80.120395,https://www.redfin.com/FL/Miami-Beach/5775-Col...,Christopher Jude,"PGutman, LLC",None,None,Christopher Jude,"PGutman, LLC",Christopher Jude,"PGutman, LLC",1,orange
1,PAST SALE,2024-09-19,Condo/Co-op,791 Crandon Blvd #707,Key Biscayne,FL,33149.0,4500000.0,4.0,5.5,OCEAN TOWER TWO CONDO,3581.0,NaN,2001.0,NaN,1257.0,5168.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11588565,N,Y,25.688337,-80.157932,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,Carlos Coto,One Sotheby's International Realty,None,None,2,blue
2,PAST SALE,2024-09-16,Condo/Co-op,1100 Biscayne Blvd #6301,Miami,FL,33132.0,4500000.0,3.0,4.5,MARQUIS CONDO,3831.0,NaN,2009.0,NaN,1175.0,4500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1100-Biscayne-...,MARMLS,A11515695,N,Y,25.785199,-80.190112,https://www.redfin.com/FL/Miami/1100-Biscayne-...,Juan Lozano,Cervera Real Estate Inc,Juan Carlos Tobon,Cervera Real Estate Inc,Adam Prince,All Prime Realty Inc,None,None,3,blue
3,PAST SALE,2024-09-16,Condo/Co-op,311 Meridian Ave #301,Miami Beach,FL,33139.0,3788000.0,3.0,3.5,LOUVER HOUSE CONDO,2224.0,NaN,2017.0,NaN,1703.0,5357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/311-Meri...,MARMLS,A11323169,N,Y,25.772507,-80.135861,https://www.redfin.com/FL/Miami-Beach/311-Meri...,Luis Devoto,London Foster Realty,None,None,Lisa Van Wagenen,Brown Harris Stevens,None,None,4,blue
4,PAST SALE,2024-09-20,Condo/Co-op,330 Sunny Isles Blvd Unit 5-2304,Sunny Isles Beach,FL,33160.0,3200000.0,4.0,4.5,PARQUE TOWERS CONDO,3084.0,NaN,2019.0,NaN,1038.0,2588.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,MARMLS,A11630528,N,Y,25.929401,-80.126023,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,Chad Carroll,"Compass Florida, LLC",None,None,Mayan Van Der Miller,"Compass Florida, LLC",None,None,5,blue
5,PAST SALE,2024-09-18,Condo/Co-op,3201 NE 183rd St #1201,Aventura,FL,33160.0,2900000.0,4.0,4.0,THE PENINSULA CONDO,3620.0,NaN,2003.0,NaN,801.0,3200.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11592370,N,Y,25.945719,-80.135657,https://www.redfin.com/FL/Aventura/3201-NE-183...,Adi Zilberberg,Gary Hennes Realtors,None,None,Tarlan Mamedov,ONE Commercial LLC,Polina Pinhasov,Yaffe International Realty,6,blue
6,PAST SALE,2024-09-16,Condo/Co-op,2900 NE 7th Ave #4602,Miami,FL,33137.0,2750000.0,4.0,5.0,BISCAYNE BEACH CONDO,2870.0,NaN,2017.0,NaN,958.0,3932.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11627938,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Barbara Elmir,One Sotheby's International Realty,Patricia Alfonso,One Sotheby's International Realty,Lianne Graubart,One Sotheby's International Realty,None,None,7,blue
7,PAST SALE,2024-09-16,Condo/Co-op,785 Crandon Blvd #503,Key Biscayne,FL,33149.0,2550000.0,3.0,4.0,CLUB TOWER TWO CONDO,2500.0,NaN,1998.0,NaN,1020.0,2507.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/785-Cra...,MARMLS,A11531824,N,Y,25.687534,-80.159408,https://www.redfin.com/FL/Key-Biscayne/785-Cra...,Mario Ojeda,Ojeda Lazar Real Estate Inc,None,None,Maria Porro Toyos,"Urdapilleta Real Estate, LLC",None,None,8,blue
8,PAST SALE,2024-09-19,Condo/Co-op,1300 Brickell Bay Dr #4201,Miami,FL,33131

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-09-16,Condo/Co-op,5775 Collins Ave #402,Miami Beach,FL,33140.0,5625000.0,2.0,3.0,AMD PL OF 2ND OCEAN FRONT,1612.0,NaN,2021.0,NaN,3489.0,4431.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5775-Col...,MARMLS,A11602237,N,Y,25.839257,-80.120395,https://www.redfin.com/FL/Miami-Beach/5775-Col...,Christopher Jude,"PGutman, LLC",None,None,Christopher Jude,"PGutman, LLC",Christopher Jude,"PGutman, LLC",1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_week_ending_09232024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'AMD PL OF 2ND OCEAN FRONT at 5775 Collins Ave #402 in Miami Beach'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-16,Condo/Co-op,5775 Collins Ave #402,Miami Beach,FL,33140.0,5625000.0,2.0,3.0,AMD PL OF 2ND OCEAN FRONT,1612.0,NaN,2021.0,NaN,3489.0,4431.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5775-Col...,MARMLS,A11602237,N,Y,25.839257,-80.120395,https://www.redfin.com/FL/Miami-Beach/5775-Col...,Christopher Jude,"PGutman, LLC",None,None,Christopher Jude,"PGutman, LLC",Christopher Jude,"PGutman, LLC",1,orange,AMD PL OF 2ND OCEAN FRONT at 5775 Collins Ave ...
1,PAST SALE,2024-09-19,Condo/Co-op,791 Crandon Blvd #707,Key Biscayne,FL,33149.0,4500000.0,4.0,5.5,OCEAN TOWER TWO CONDO,3581.0,NaN,2001.0,NaN,1257.0,5168.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11588565,N,Y,25.688337,-80.157932,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,Carlos Coto,One Sotheby's International Realty,None,None,2,blue,OCEAN TOWER TWO CONDO at 791 Crandon Blvd #707...
2,PAST SALE,2024-09-16,Condo/Co-op,1100 Biscayne Blvd #6301,Miami,FL,33132.0,4500000.0,3.0,4.5,MARQUIS CONDO,3831.0,NaN,2009.0,NaN,1175.0,4500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1100-Biscayne-...,MARMLS,A11515695,N,Y,25.785199,-80.190112,https://www.redfin.com/FL/Miami/1100-Biscayne-...,Juan Lozano,Cervera Real Estate Inc,Juan Carlos Tobon,Cervera Real Estate Inc,Adam Prince,All Prime Realty Inc,None,None,3,blue,MARQUIS CONDO at 1100 Biscayne Blvd #6301 in M...
3,PAST SALE,2024-09-16,Condo/Co-op,311 Meridian Ave #301,Miami Beach,FL,33139.0,3788000.0,3.0,3.5,LOUVER HOUSE CONDO,2224.0,NaN,2017.0,NaN,1703.0,5357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/311-Meri...,MARMLS,A11323169,N,Y,25.772507,-80.135861,https://www.redfin.com/FL/Miami-Beach/311-Meri...,Luis Devoto,London Foster Realty,None,None,Lisa Van Wagenen,Brown Harris Stevens,None,None,4,blue,LOUVER HOUSE CONDO at 311 Meridian Ave #301 in...
4,PAST SALE,2024-09-20,Condo/Co-op,330 Sunny Isles Blvd Unit 5-2304,Sunny Isles Beach,FL,33160.0,3200000.0,4.0,4.5,PARQUE TOWERS CONDO,3084.0,NaN,2019.0,NaN,1038.0,2588.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,MARMLS,A11630528,N,Y,25.929401,-80.126023,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,Chad Carroll,"Compass Florida, LLC",None,None,Mayan Van Der Miller,"Compass Florida, LLC",None,None,5,blue,PARQUE TOWERS CONDO at 330 Sunny Isles Blvd Un...
5,PAST SALE,2024-09-18,Condo/Co-op,3201 NE 183rd St #1201,Aventura,FL,33160.0,2900000.0,4.0,4.0,THE PENINSULA CONDO,3620.0,NaN,2003.0,NaN,801.0,3200.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11592370,N,Y,25.945719,-80.135657,https://www.redfin.com/FL/Aventura/3201-NE-183...,Adi Zilberberg,Gary Hennes Realtors,None,None,Tarlan Mamedov,ONE Commercial LLC,Polina Pinhasov,Yaffe International Realty,6,blue,THE PENINSULA CONDO at 3201 NE 183rd St #1201 ...
6,PAST SALE,2024-09-16,Condo/Co-op,2900 NE 7th Ave #4602,Miami,FL,33137.0,2750000.0,4.0,5.0,BISCAYNE BEACH CONDO,2870.0,NaN,2017.0,NaN,958.0,3932.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11627938,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Barbara Elmir,One Sotheby's International Realty,Patricia Alfonso,One Sotheby's International Realty,Lianne Graubart,One Sotheby's International Realty,None,None,7,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #4602 ...
7,PAST SALE,2024-09-16,Condo/Co-op,785 Crandon Blvd #503,Key Biscayne,FL,33149.0,2550000.0,3.0,4.0,CLUB TOWER TWO CONDO,2500.0,NaN,1998.0,NaN,1

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: AMD PL OF 2ND OCEAN FRONT closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,850,000M to $5,625,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami, Key Biscayne, Miami Beach, Sunny Isles Beach, Aventura

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 134 condo sales totaling $91,741,989 million from Sep. 15th to Sep. 22nd. The previous week, brokers closed 107 condo sales totaling $93,800,000.

Last week’s units sold for an average of $684,642, lower than the $876,980 average price from the previous week. T

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-16,Condo/Co-op,5775 Collins Ave #402,Miami Beach,FL,33140.0,5625000.0,2.0,3.0,AMD PL OF 2ND OCEAN FRONT,1612.0,NaN,2021.0,NaN,3489.0,4431.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5775-Col...,MARMLS,A11602237,N,Y,25.839257,-80.120395,https://www.redfin.com/FL/Miami-Beach/5775-Col...,Christopher Jude,"PGutman, LLC",None,None,Christopher Jude,"PGutman, LLC",Christopher Jude,"PGutman, LLC",1,orange,AMD PL OF 2ND OCEAN FRONT at 5775 Collins Ave ...
1,PAST SALE,2024-09-19,Condo/Co-op,791 Crandon Blvd #707,Key Biscayne,FL,33149.0,4500000.0,4.0,5.5,OCEAN TOWER TWO CONDO,3581.0,NaN,2001.0,NaN,1257.0,5168.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11588565,N,Y,25.688337,-80.157932,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,Victor Florencio Sanchez,"Ocean Club Realty, Inc.",None,None,Carlos Coto,One Sotheby's International Realty,None,None,2,blue,OCEAN TOWER TWO CONDO at 791 Crandon Blvd #707...
2,PAST SALE,2024-09-16,Condo/Co-op,1100 Biscayne Blvd #6301,Miami,FL,33132.0,4500000.0,3.0,4.5,MARQUIS CONDO,3831.0,NaN,2009.0,NaN,1175.0,4500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1100-Biscayne-...,MARMLS,A11515695,N,Y,25.785199,-80.190112,https://www.redfin.com/FL/Miami/1100-Biscayne-...,Juan Lozano,Cervera Real Estate Inc,Juan Carlos Tobon,Cervera Real Estate Inc,Adam Prince,All Prime Realty Inc,None,None,3,blue,MARQUIS CONDO at 1100 Biscayne Blvd #6301 in M...
3,PAST SALE,2024-09-16,Condo/Co-op,311 Meridian Ave #301,Miami Beach,FL,33139.0,3788000.0,3.0,3.5,LOUVER HOUSE CONDO,2224.0,NaN,2017.0,NaN,1703.0,5357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/311-Meri...,MARMLS,A11323169,N,Y,25.772507,-80.135861,https://www.redfin.com/FL/Miami-Beach/311-Meri...,Luis Devoto,London Foster Realty,None,None,Lisa Van Wagenen,Brown Harris Stevens,None,None,4,blue,LOUVER HOUSE CONDO at 311 Meridian Ave #301 in...
4,PAST SALE,2024-09-20,Condo/Co-op,330 Sunny Isles Blvd Unit 5-2304,Sunny Isles Beach,FL,33160.0,3200000.0,4.0,4.5,PARQUE TOWERS CONDO,3084.0,NaN,2019.0,NaN,1038.0,2588.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,MARMLS,A11630528,N,Y,25.929401,-80.126023,https://www.redfin.com/FL/Sunny-Isles-Beach/33...,Chad Carroll,"Compass Florida, LLC",None,None,Mayan Van Der Miller,"Compass Florida, LLC",None,None,5,blue,PARQUE TOWERS CONDO at 330 Sunny Isles Blvd Un...
5,PAST SALE,2024-09-18,Condo/Co-op,3201 NE 183rd St #1201,Aventura,FL,33160.0,2900000.0,4.0,4.0,THE PENINSULA CONDO,3620.0,NaN,2003.0,NaN,801.0,3200.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11592370,N,Y,25.945719,-80.135657,https://www.redfin.com/FL/Aventura/3201-NE-183...,Adi Zilberberg,Gary Hennes Realtors,None,None,Tarlan Mamedov,ONE Commercial LLC,Polina Pinhasov,Yaffe International Realty,6,blue,THE PENINSULA CONDO at 3201 NE 183rd St #1201 ...
6,PAST SALE,2024-09-16,Condo/Co-op,2900 NE 7th Ave #4602,Miami,FL,33137.0,2750000.0,4.0,5.0,BISCAYNE BEACH CONDO,2870.0,NaN,2017.0,NaN,958.0,3932.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11627938,N,Y,25.805017,-80.185768,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,Barbara Elmir,One Sotheby's International Realty,Patricia Alfonso,One Sotheby's International Realty,Lianne Graubart,One Sotheby's International Realty,None,None,7,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #4602 ...
7,PAST SALE,2024-09-16,Condo/Co-op,785 Crandon Blvd #503,Key Biscayne,FL,33149.0,2550000.0,3.0,4.0,CLUB TOWER TWO CONDO,2500.0,NaN,1998.0,NaN,1

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: AMD PL OF 2ND OCEAN FRONT closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,850,000M to $5,625,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami, Key Biscayne, Miami Beach, Sunny Isles Beach, Aventura

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 134 condo sales totaling $91,741,989 million from Sep. 15th to Sep. 22nd. The previous week, brokers closed 107 condo sales totaling $93,800,000.

Last week’s units sold for an average of $684,642, lower than the $876,980 average price from the previous week. T

In [48]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami-Beach/5775-Collins-Ave-33140/unit-402/home/42774988


In [55]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Key-Biscayne/791-Crandon-Blvd-33149/unit-707/home/43276386


In [56]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Aventura/3201-NE-183rd-St-33160/unit-2907/home/43307682


In [57]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/1300-Brickell-Bay-Dr-33131/unit-4201/home/55272944


In [58]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/5775-Collins-Ave-33140/unit-402/home/42774988


In [59]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/3201-NE-183rd-St-33160/unit-2907/home/43307682


In [56]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-09-12,Condo/Co-op,15701 Collins Ave #2205,Sunny Isles Beach,FL,33160.0,7100000.0,4.0,5.5,15701 COLLINS CONDO,3204.0,NaN,2020.0,NaN,2216.0,5457.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11596165,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Catherine Kalifeh,"Compass Florida, LLC.",Frank Lustig,"Compass Florida, LLC.",None,None,None,None,1,orange,15701 COLLINS CONDO at 15701 Collins Ave #2205...
1,PAST SALE,2024-09-12,Condo/Co-op,2655 S Bayshore Dr #801,Miami,FL,33133.0,6554000.0,4.0,3.5,Mr. C Residences,2715.0,NaN,2024.0,NaN,2414.0,6936.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11616843,N,Y,25.731196,-80.236270,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,Sari Shapiro,Douglas Elliman,None,None,Mayan Van Der Miller,"Compass Florida, LLC",None,None,2,blue,Mr. C Residences at 2655 S Bayshore Dr #801 in...
2,PAST SALE,2024-09-09,Condo/Co-op,18555 Collins Ave #4803,Sunny Isles Beach,FL,33160.0,4200000.0,3.0,4.5,18555 COLLINS AVENUE COND,3130.0,NaN,2016.0,NaN,1342.0,6171.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11621758,N,Y,25.948195,-80.120410,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Melissa Barragan,Dezer Platinum Realty LLC,None,None,Todd Nepola,Current Capital Realty,None,None,3,blue,18555 COLLINS AVENUE COND at 18555 Collins Ave...
3,PAST SALE,2024-09-10,Condo/Co-op,16051 Collins Ave #3103,Sunny Isles Beach,FL,33160.0,3900000.0,4.0,5.5,TURNBERRY OCEAN COLONY,3480.0,NaN,2006.0,NaN,1121.0,6008.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11556954,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Jackson Keddell,Douglas Elliman,None,None,Alisa Gitman,"Miami Matrix Realty, LLC.",None,None,4,blue,TURNBERRY OCEAN COLONY at 16051 Collins Ave #3...
4,PAST SALE,2024-09-10,Condo/Co-op,2669 S Bayshore Dr Unit 802N,Miami,FL,33133.0,3400000.0,3.0,3.5,DINNER KEY,1833.0,NaN,2016.0,NaN,1855.0,3090.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11627841,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",Tarlan Mamedov,ONE Commercial LLC,None,None,5,blue,DINNER KEY at 2669 S Bayshore Dr Unit 802N in ...
5,PAST SALE,2024-09-11,Condo/Co-op,16051 Collins Ave #2001,Sunny Isles Beach,FL,33160.0,3380000.0,3.0,4.5,TURNBERRY OCEAN COLONY NO,2775.0,NaN,2006.0,NaN,1218.0,5417.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11604152,N,Y,25.926219,-80.121522,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Alisa Gitman,"Miami Matrix Realty, LLC.",Oksana Ivanenko,"Miami Matrix Realty, LLC.",Uri Sefchovich,The Keyes Company,None,None,6,blue,TURNBERRY OCEAN COLONY NO at 16051 Collins Ave...
6,PAST SALE,2024-09-12,Condo/Co-op,18975 Collins Ave #4004,Sunny Isles Beach,FL,33160.0,3250000.0,2.0,2.5,18975 COLLINS CONDO,1688.0,NaN,2019.0,NaN,1925.0,2774.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11444654,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Jorge Llano,Dezer Platinum Realty LLC,None,None,geanemiami Brito,"Compass Florida, LLC.",None,None,7,blue,18975 COLLINS CONDO at 18975 Collins Ave #4004...
7,PAST SALE,2024-09-09,Condo/Co-op,848 Brickell Key Dr #1901,Miami,FL,33131.0,2260000.0,3.0,2.5,THREE TEQUESTA POINT COND,2353.0,NaN,2001.0,NaN,960.0,2338.0,Sold,NaN,NaN,https://www.

## Time on Market Calculator

In [61]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 6, 16) ## List (Earlier) date
date2 = datetime(2024, 9, 16) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

92


## Clean CSV for Datawrapper Chart

In [62]:
chart_df = df_top_ten

In [63]:
chart_df = chart_df.fillna(" ")

In [64]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [65]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [66]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [67]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [68]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [69]:
csv_date_string = today.strftime("%m_%d_%Y")

In [70]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")